In [43]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv('../Data/subredditdata.csv')

In [45]:
df.head()

,title,selftext,subreddit,created_utc
0,Do the crazy customers realize that they becom...,[removed],TalesFromRetail,1636729943
1,Store music volume controls,"My most recent job, and hopefully last, in ret...",TalesFromRetail,1636729678
2,Ways of Rejection,I'll just hop right into it as there's no poin...,TalesFromRetail,1636725427
3,JUST TELL US WHAT IT'S USED FOR!,[removed],TalesFromRetail,1636694714
4,You've got to go collect stray shopping carts ...,[removed],TalesFromRetail,1636679308


### Dropping NaN from data set

In [46]:
df.isnull().sum()

title           0
selftext       65
subreddit       0
created_utc     0
dtype: int64

In [47]:
df[df['selftext'].isnull()].value_counts(df['subreddit']) # of rows for each with blank self texts

subreddit
TalesFromRetail         41
talesfromtechsupport    24
dtype: int64

In [48]:
df = df.dropna() # all 65 na rows are dropped now

In [49]:
df['subreddit'].value_counts()

talesfromtechsupport    1470
TalesFromRetail         1459
Name: subreddit, dtype: int64

In [50]:
df['subreddit'].value_counts(normalize=True) # distribution still almost 50/50

talesfromtechsupport    0.501878
TalesFromRetail         0.498122
Name: subreddit, dtype: float64

### Checking the dtypes and setting subreddit column to binary

In [51]:
df.dtypes

title          object
selftext       object
subreddit      object
created_utc     int64
dtype: object

### TalesFromRetail = 0, TalesFromTechSupport = 1

In [52]:
df['subreddit'] = df['subreddit'].map({'TalesFromRetail': 0, 'talesfromtechsupport': 1})

In [53]:
df.head(2)

,title,selftext,subreddit,created_utc
0,Do the crazy customers realize that they becom...,[removed],0,1636729943
1,Store music volume controls,"My most recent job, and hopefully last, in ret...",0,1636729678


### Furthur Exploration

In [54]:
df['selftext'].value_counts() # 1503 of selftext is removed, may add them as a stopword

[removed]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

### Going to change [removed] to just removed_asdf_fix in our DF

In [55]:
def selftext_fix(self_cell):
    if self_cell == '[removed]':
        return 'removed_asdf_fix' # setting as this string since I don't want an accidental replace on something else
    else:
        return self_cell

In [56]:
df['selftext'] = df['selftext'].map(selftext_fix)

In [57]:
df.head()

,title,selftext,subreddit,created_utc
0,Do the crazy customers realize that they becom...,removed_asdf_fix,0,1636729943
1,Store music volume controls,"My most recent job, and hopefully last, in ret...",0,1636729678
2,Ways of Rejection,I'll just hop right into it as there's no poin...,0,1636725427
3,JUST TELL US WHAT IT'S USED FOR!,removed_asdf_fix,0,1636694714
4,You've got to go collect stray shopping carts ...,removed_asdf_fix,0,1636679308


In [58]:
df.groupby('subreddit').nunique()

,title,selftext,created_utc
subreddit,,,
0,1428,550,1459
1,1453,866,1469


In [59]:
df.to_csv('../Data/subredditdatacleaned.csv')